################Topic generation:::Preprocess#################################################

In [1]:
#Get the data
import os
import importlib
import pandas as pd
google_data = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_eng_google.csv", encoding='utf-8-sig')['content']
apple_data = pd.read_csv("data"+os.sep+"part"+os.sep+"Suomi112_eng_apple.csv", encoding='utf-8-sig')['content']

In [2]:
data = pd.concat([google_data,apple_data])

In [9]:
import utils
importlib.reload(utils)
processor = utils.Processor()
processor.ini_dowload()

[nltk_data] Downloading package stopwords to data/lib/stopwords...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to data/lib/wordnet...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to data/lib/omw...


In [10]:
data = data.apply(lambda x: processor.preprocess(str(x)))
data = data.apply(lambda x: processor.tokenize(x))
data = data.apply(lambda x: processor.remove_stopwords(x))
data = data.apply(lambda x: processor.process_tokens(x))
data
data_copy = data

################Topic generation:::Topics#################################################
Mostly taken from https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24

In [11]:
flat_data = [tag for sentence in data for tag in sentence ]
from gensim.corpora import Dictionary
dictionary = Dictionary(data)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in data]

In [12]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1

0 also
1 could
2 danger
3 emerg
4 help
5 inform
6 necessari
7 need
8 night
9 option
10 program
11 situat
12 user
13 ask
14 call
15 fortun
16 know
17 number
18 phone
19 open
20 screen
21 posit
22 thumb
23 address
24 real
25 use
26 good
27 chang
28 delet
29 set
30 thing
31 time
32 applic
33 case
34 import
35 instal
36 work
37 featur
38 great
39 idea
40 rememb
41 still
42 would
43 find
44 lose
45 realli
46 automat
47 center
48 coordin
49 locat
50 send
51 crash
52 abl
53 alarm
54 even
55 keep
56 live
57 make
58 notif
59 peopl
60 think
61 though
62 well
63 data
64 face
65 alreadi
66 first
67 everi
68 cannot
69 android
70 notic
71 person
72 servic
73 easi
74 announc
75 constant
76 go
77 otherwis
78 traffic
79 turn
80 life
81 definit
82 mani
83 worth
84 recommend
85 safe
86 alway
87 happen
88 someth
89 quick
90 experi
91 accid
92 like
93 remov
94 seem
95 finland
96 take
97 download
98 updat
99 give
100 save
101 possibl
102 probabl
103 quit
104 nice
105 button
106 come
107 immedi
108 requir
10

In [13]:
bow_doc_4310 = bow_corpus[1]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],dictionary[bow_doc_4310[i][0]],bow_doc_4310[i][1]))

Word 13 ("ask") appears 1 time.
Word 14 ("call") appears 2 time.
Word 15 ("fortun") appears 1 time.
Word 16 ("know") appears 1 time.
Word 17 ("number") appears 2 time.
Word 18 ("phone") appears 1 time.


In [14]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.18491882757277014),
 (1, 0.5426701729455495),
 (2, 0.24380901457731965),
 (3, 0.11672794900115502),
 (4, 0.15611909700179866),
 (5, 0.3719453348291787),
 (6, 0.1892577979494741),
 (7, 0.15248049327141402),
 (8, 0.2534977241247234),
 (9, 0.24685725909141704),
 (10, 0.16715493941001502),
 (11, 0.4220019639351989),
 (12, 0.21268405590801284)]


In [15]:
import gensim
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.099*"call" + 0.065*"applic" + 0.062*"emerg" + 0.034*"locat" + 0.034*"center" + 0.032*"number" + 0.030*"phone" + 0.021*"help" + 0.018*"use" + 0.017*"inform"
Topic: 1 
Words: 0.070*"import" + 0.044*"phone" + 0.042*"applic" + 0.041*"program" + 0.041*"realli" + 0.040*"number" + 0.039*"save" + 0.032*"would" + 0.029*"need" + 0.024*"also"
Topic: 2 
Words: 0.093*"work" + 0.037*"applic" + 0.032*"emerg" + 0.031*"call" + 0.029*"well" + 0.029*"phone" + 0.028*"use" + 0.028*"start" + 0.027*"realli" + 0.025*"make"
Topic: 3 
Words: 0.134*"good" + 0.058*"batteri" + 0.055*"applic" + 0.046*"locat" + 0.039*"use" + 0.031*"background" + 0.021*"call" + 0.021*"would" + 0.019*"emerg" + 0.017*"even"
Topic: 4 
Words: 0.041*"emerg" + 0.036*"updat" + 0.034*"great" + 0.029*"good" + 0.028*"time" + 0.025*"notif" + 0.024*"download" + 0.024*"program" + 0.024*"phone" + 0.021*"even"
Topic: 5 
Words: 0.092*"phone" + 0.061*"number" + 0.039*"good" + 0.036*"work" + 0.034*"oneplus" + 0.031*"applic" + 0.029*

In [16]:
import gensim
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.093*"import" + 0.043*"help" + 0.041*"good" + 0.040*"posit" + 0.039*"download" + 0.032*"idea" + 0.027*"hope" + 0.019*"right" + 0.018*"applic" + 0.016*"need"
Topic: 1 Word: 0.038*"oneplus" + 0.037*"phone" + 0.035*"work" + 0.034*"thing" + 0.028*"turn" + 0.027*"crash" + 0.027*"open" + 0.026*"thank" + 0.026*"applic" + 0.025*"program"
Topic: 2 Word: 0.050*"safe" + 0.037*"applic" + 0.033*"good" + 0.032*"find" + 0.030*"download" + 0.027*"start" + 0.025*"announc" + 0.023*"tri" + 0.022*"nice" + 0.022*"locat"
Topic: 3 Word: 0.084*"work" + 0.051*"face" + 0.042*"know" + 0.036*"smile" + 0.036*"need" + 0.030*"never" + 0.028*"help" + 0.028*"case" + 0.021*"locat" + 0.020*"phone"
Topic: 4 Word: 0.055*"use" + 0.037*"locat" + 0.030*"emerg" + 0.029*"call" + 0.028*"applic" + 0.026*"work" + 0.023*"center" + 0.022*"coordin" + 0.021*"batteri" + 0.021*"everyon"
Topic: 5 Word: 0.074*"necessari" + 0.035*"emerg" + 0.035*"thumb" + 0.030*"good" + 0.029*"know" + 0.027*"realli" + 0.026*"phone" + 0.024

In [17]:
for index, score in sorted(lda_model_tfidf[bow_corpus[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.47235599160194397	 
Topic: 0.065*"number" + 0.037*"time" + 0.035*"save" + 0.027*"easier" + 0.027*"phone" + 0.027*"emerg" + 0.021*"help" + 0.020*"finland" + 0.019*"ask" + 0.019*"person"

Score: 0.43873053789138794	 
Topic: 0.055*"use" + 0.037*"locat" + 0.030*"emerg" + 0.029*"call" + 0.028*"applic" + 0.026*"work" + 0.023*"center" + 0.022*"coordin" + 0.021*"batteri" + 0.021*"everyon"

Score: 0.011116859503090382	 
Topic: 0.084*"work" + 0.051*"face" + 0.042*"know" + 0.036*"smile" + 0.036*"need" + 0.030*"never" + 0.028*"help" + 0.028*"case" + 0.021*"locat" + 0.020*"phone"

Score: 0.011116379871964455	 
Topic: 0.074*"necessari" + 0.035*"emerg" + 0.035*"thumb" + 0.030*"good" + 0.029*"know" + 0.027*"realli" + 0.026*"phone" + 0.024*"call" + 0.022*"lose" + 0.020*"applic"

Score: 0.011113779619336128	 
Topic: 0.050*"safe" + 0.037*"applic" + 0.033*"good" + 0.032*"find" + 0.030*"download" + 0.027*"start" + 0.025*"announc" + 0.023*"tri" + 0.022*"nice" + 0.022*"locat"

Score: 0.011113712564